In [48]:
import pandas as pd
import plotly.express as px


df = pd.read_csv('../data/processed/livros_limpos.csv')

df.head()

,title,authors,categories,averageRating,ratingsCount,pageCount,publishedDate,description,price,currency,isEbook,published_year,book_size
0,Must Read: Rediscovering American Bestsellers,Sarah Churchwell,Literary Criticism,0.0,0,284,2012-08-02,What is it about certain books that makes them...,229.63,BRL,True,2012,medium
1,Focus On: 100 Most Popular Fantasy Anime and M...,Wikipedia contributors,Outros,0.0,0,1062,NaN,Sem descrição,0.00,NaN,False,NaN,long
2,Best Sellers,Desconhecido,American literature,0.0,0,448,1983,Sem descrição,0.00,NaN,False,1983,long
3,Data Preparation for Data Mining,Dorian Pyle,Computers,3.0,3,566,1999-03-22,"This book focuses on the importance of clean, ...",0.00,NaN,False,1999,long
4,A Richard Wright Bibliography,Kenneth Kinnamon,Social Science,0.0,0,1000,1988-01-13,Any future biographical work on Richard Wright...,397.53,BRL,True,1988,long


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           160 non-null    object 
 1   authors         160 non-null    object 
 2   categories      160 non-null    object 
 3   averageRating   160 non-null    float64
 4   ratingsCount    160 non-null    int64  
 5   pageCount       160 non-null    int64  
 6   publishedDate   159 non-null    object 
 7   description     160 non-null    object 
 8   price           160 non-null    float64
 9   currency        16 non-null     object 
 10  isEbook         160 non-null    bool   
 11  published_year  159 non-null    object 
 12  book_size       156 non-null    object 
dtypes: bool(1), float64(2), int64(2), object(8)
memory usage: 15.3+ KB


## 1. Distribuição de Formato: Digital vs. Físico

Nesta primeira análise, verificamos a predominância dos formatos de leitura na amostra coletada. O objetivo é entender se o nicho de **Investing** (ou o tema escolhido) possui uma oferta maior de e-books ou se ainda mantém uma base forte de livros físicos indexados.

In [62]:
df_ebook = df['isEbook'].value_counts().reset_index()
df_ebook.columns = ['Tipo', 'Total']
df_ebook['Tipo'] = df_ebook['Tipo'].map({True: 'E-book', False: 'Físico/Outros'})

fig = px.bar(df_ebook, 
             y='Total', 
             x='Tipo', 
             title='Distribuição de Formato: E-book vs Físico',
             labels={'Total': 'Quantidade', 'Tipo': 'Formato'},
             color='Tipo')
fig.show()

## 2. Perfil de Extensão e Volumetria Editorial

Nesta seção, analisamos como os livros estão distribuídos em termos de volume de páginas. Utilizamos a classificação criada na etapa de transformação para agrupar os títulos em: **Short** (até 200 páginas), **Medium** (201 a 400) e **Long** (acima de 400).

A tabela abaixo detalha as métricas estatísticas de cada grupo, enquanto o gráfico de barras permite visualizar rapidamente qual perfil de livro predomina no tema escolhido.

In [63]:
tabela_resumo = df.groupby('book_size', observed=True)['pageCount'].agg(['mean', 'min', 'max', 'count']).reset_index()
tabela_resumo.columns = ['Tamanho', 'Média de Páginas', 'Mínimo', 'Máximo', 'Total de Livros']
display(tabela_resumo)

,Tamanho,Média de Páginas,Mínimo,Máximo,Total de Livros
0,long,898.061947,404,2795,113
1,medium,290.125000,204,382,24
2,short,92.578947,38,199,19


In [69]:
df_size = df.dropna(subset=['book_size'])

fig = px.bar(df_size['book_size'].value_counts().reset_index(), 
             x='book_size', 
             y='count',
             title='Distribuição de Tamanho dos Livros',
             labels={'book_size': 'Tamanho', 'count': 'Quantidade'},
             color='book_size')

fig.show()

## 3. Evolução Temporal das Publicações

Abaixo, utilizamos um histograma para visualizar a distribuição dos livros ao longo dos anos. Esta análise é fundamental para identificar se a base de dados reflete tendências recentes do mercado ou se é composta por títulos clássicos e históricos. 

*Nota: Valores ausentes na data de publicação foram desconsiderados para garantir a precisão do eixo temporal.*

In [65]:
df['published_year'] = pd.to_numeric(df['published_year'], errors='coerce')

fig_year = px.histogram(df.dropna(subset=['published_year']), 
                        x='published_year',
                        title='Distribuição de Livros por Ano de Publicação',
                        labels={'published_year': 'Ano'},
                        nbins=20)

fig_year.update_layout(bargap=0.1)
fig_year.show()

## 4. Destaques do Catálogo: Os Maiores Livros

Para complementar a análise estatística, listamos abaixo os 10 títulos com o maior número de páginas na amostra atual. Esta visualização ajuda a identificar obras de referência, enciclopédias ou manuais extensos que se destacam pela densidade de conteúdo em relação à média do mercado.

In [66]:
print("Top 10 maiores livros")
display(df[['title','categories', 'authors', 'pageCount']].sort_values(by='pageCount', ascending=False).head(10))

Top 10 maiores livros


,title,categories,authors,pageCount
127,The Number One New York Times Best Seller,Literary Criticism,John Bear,2795
90,Book Review Digest,Bibliography,Desconhecido,2284
34,Books In Print 2001-2002,Literary Criticism,R R Bowker Publishing,2206
25,The Education Index,Education,Desconhecido,2060
52,The Atlantic Monthly,American essays,Desconhecido,1640
27,Business Week,Business,Desconhecido,1626
102,The Christian Advocate,Methodist Church,Desconhecido,1624
114,Literary Digest,American wit and humor,Desconhecido,1598
69,The Athenaeum,England,Desconhecido,1468
10,The Retail Bookseller,American literature,Desconhecido,1452


## 5. Análise de Segmentação: Principais Gêneros e Temas

Abaixo, exploramos a classificação dos livros por categorias. Esta visualização revela a diversidade de subtemas presentes na amostra de **Investing**. 

> **Nota Técnica:** Observou-se uma alta fragmentação nos metadados de categorias fornecidos pela API, com uma grande variedade de termos específicos. O gráfico abaixo consolida os 10 gêneros mais frequentes, permitindo identificar quais áreas correlatas (como Negócios, Economia ou Finanças Pessoais) possuem maior representatividade no catálogo.

In [67]:
top_categorias = df['categories'].value_counts().head(10).reset_index()
top_categorias.columns = ['Categoria', 'Total de Livros']

fig_cat = px.bar(top_categorias, 
                 x='Total de Livros', 
                 y='Categoria', 
                 title='Top 10 Categorias que Mais Aparecem',
                 orientation='h',
                 color='Total de Livros',
                 color_continuous_scale='Blues',
                 text_auto=True)
fig_cat.update_layout(yaxis={'categoryorder':'total ascending'})

fig_cat.show()

## 6. Comparativo de Extensão por Formato (Box Plot)

Para aprofundar a compreensão sobre o catálogo, utilizamos um **Box Plot** para comparar a distribuição do número de páginas entre e-books e livros físicos. 

Esta visualização é valiosa pois nos permite observar:
* **Mediana:** O valor central de páginas para cada formato.
* **Dispersão:** Se um formato apresenta maior variação de tamanho que o outro.
* **Outliers:** Livros com número de páginas muito acima ou abaixo da média (representados pelos pontos individuais).

Este gráfico ajuda a validar se e-books tendem a ser obras mais curtas ou se mantêm a mesma densidade dos livros tradicionais.

In [71]:
fig_box = px.box(df, 
                 x='isEbook', 
                 y='pageCount', 
                 title='Distribuição de Páginas: E-book vs Físico',
                 labels={'isEbook': 'É E-book?', 'pageCount': 'Número de Páginas'},
                 color='isEbook',
                 points="all")
fig_box.show()